# TRANSACTION RISK

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import re
import os
from datetime import datetime
from pprint import pprint
from configparser import ConfigParser
import pyodbc

In [ ]:
pwd

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
#config.read(r'config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
cursor.execute("""INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,
                  TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE) 
                  VALUES(COALESCE((SELECT (MAX(JOB_RUN_ID)+1) FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN),1), 
                  'TRANSACTION_SUMMARY_STG_PROCESSING',GETDATE(),'ERROR','RISK_TRANSACTION_SUMMARY_STG','CPS_REPORT_DETAILS',
                  (SELECT MAX(UPDATE_DATE) FROM FINCORE_DB.FINCORE.CPS_REPORT_DETAILS))""")
conn.commit()

In [ ]:
job_id = pd.read_sql_query("SELECT MAX(JOB_RUN_ID) AS JOB_RUN_ID FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN WHERE JOB_NAME = 'TRANSACTION_SUMMARY_STG_PROCESSING' ",conn)
job_id = job_id.iloc[0,0]
%store job_id

In [ ]:
no_of_records = pd.read_sql_query('''SELECT COUNT(1) FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL (NOLOCK)''' ,conn)
no_of_records = no_of_records.iloc[0,0]

## Populating RISK_TRANSACTION_SUMMARY_STG

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
tsone='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT 
CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,'TS1' AS TRAN_TYPE,TS1.TRANSACTIONTYPE AS CR_DR_DEP_WITH,TS1.TRANSACTIONNO AS TRANSACTION_ID,
CAST(TS1.TRANSACTIONDATE AS DATE) AS TRAN_DATE,TS1.AMOUNT AS TRAN_AMNT,
CASE WHEN TS1.TRANSACTIONTYPE = 'Credit' THEN ADMB.ORIGINAL_PINCODE ELSE ADMS.ORIGINAL_PINCODE END  AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,GETDATE() AS CREATE_DATE,GETDATE() AS UPDATE_DATE
FROM [FINCORE_DB].[FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSONE TS1 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS1.REPORTID
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND COALESCE(TS1.SENDERACNO,TS1.BENEFICIARYACNO) = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID
LEFT JOIN 
(SELECT REF.* FROM (
SELECT IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY IFSC_CODE ORDER BY IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1
) AS ADMS ON TS1.SENDERIFSC = ADMS.IFSC_CODE
LEFT JOIN 
(SELECT REF.* FROM (
SELECT IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY IFSC_CODE ORDER BY IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1
) AS ADMB ON TS1.BENEFICIARYIFSC = ADMB.IFSC_CODE;'''

##tc2 population
tctwo='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,
CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,
AD.ID_,
'TC2' AS TRAN_TYPE,
CASE WHEN TC2.DEPOSITEWITHDRAWL='DEPOSIT' THEN 'Credit' WHEN TC2.DEPOSITEWITHDRAWL='WITHDRAWAL' THEN 'Debit' END AS DEPOSITEWITHDRAWL,
TC2.TRANSACTIONID,
CAST(TC2.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
TC2.AMOUNT AS TRAN_AMNT,
ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE FROM 
[FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TCTWO TC2 WITH (NOLOCK) ON CPSRD.REPORT_ID = TC2.REPORTID
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND TC2.ACCOUNTNUMBER = AD.ACCOUNTNUMBER 
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID
LEFT JOIN 
(
SELECT REF.* FROM (
SELECT A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.BRANCH_CODE ORDER BY A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1) AS ADM ON TC2.TXNBRANCHCODE = ADM.BRANCH_CODE;'''

##ts3 population
tsthree='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,
CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,
AD.ID_,
'TS3' AS TRAN_TYPE,
TS3.TRANSACTIONTYPE AS CR_DR_DEP_WITH,
TS3.TRANSACTIONID,
CAST(TS3.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
TS3.TRANSACTIONAMOUNT AS TRAN_AMNT,
CASE WHEN TS3.TRANSACTIONTYPE = 'Credit' THEN ADMB.ORIGINAL_PINCODE ELSE ADMS.ORIGINAL_PINCODE END  AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE
FROM 
[FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSTHREE TS3 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS3.REPORTID
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND COALESCE(TS3.SENDERACNO,TS3.BENEFICIARYACNO) = AD.ACCOUNTNUMBER 
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID
LEFT JOIN 
(SELECT REF.* FROM (SELECT IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY IFSC_CODE ORDER BY IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1) AS ADMS ON TS3.SENDERIFSC = ADMS.IFSC_CODE
LEFT JOIN 
(
SELECT REF.* FROM (SELECT IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY IFSC_CODE ORDER BY IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1) AS ADMB ON TS3.BENEFICIARYIFSC = ADMB.IFSC_CODE;'''

##tc1 population
tcone='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,'TC1' AS TRAN_TYPE,
CASE WHEN TC1.DEPOSITWITHDRAWAL='DEPOSIT' THEN 'Credit' WHEN TC1.DEPOSITWITHDRAWAL='WITHDRAWAL' THEN 'Debit' END AS DEPOSITEWITHDRAWL,
TC1.TRANSACTIONID,CAST(TC1.TRANSACTIONDATE AS DATE) AS TRAN_DATE, TC1.TRANSACTIONAMOUNT AS TRAN_AMNT,
COALESCE(ADM1.ORIGINAL_PINCODE,ADM2.ORIGINAL_PINCODE) AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,GETDATE() AS UPDATE_DATE FROM 
[FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN  FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TCONE TC1  WITH (NOLOCK) ON  CPSRD.REPORT_ID = TC1.REPORTID
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND TC1.ACCOUNTNUMBER = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID
LEFT JOIN 
(
SELECT REF.* FROM (SELECT A.ATM_ID, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.ATM_ID ORDER BY A.ATM_ID, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.ATM A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1
) AS ADM1 ON TC1.ATMCAMID = ADM1.ATM_ID
LEFT JOIN 
(SELECT REF.* FROM (SELECT A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.BRANCH_CODE ORDER BY A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1) AS ADM2 ON TC1.BRANCHCODE = ADM2.BRANCH_CODE;'''

##cb1 population

cbone='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,'CB1' AS TRAN_TYPE,NULL AS CR_DR_DEP_WITH,CB1.TRANSACTIONID,
CAST(CB1.TRANSACTIONDATE AS DATE) AS TRAN_DATE,CB1.AMOUNTININR AS TRAN_AMNT,
ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,GETDATE() AS UPDATE_DATE 
FROM [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_CBWTRBANK AS CB1 WITH (NOLOCK) ON  CPSRD.REPORT_ID = CB1.REPORTID
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
LEFT JOIN FINCORE_DB.FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT AS PMCFR WITH (NOLOCK) ON CB1.REPORTID = PMCFR.REPORT_ID
LEFT JOIN FINCORE_DB.FINCORE.PERSON_MASTER_ADDRESS_FINGATE_REPORT AS PMAFR WITH (NOLOCK) ON CB1.REPORTID = PMAFR.REPORT_ID AND PMCFR.PERSON_MASTER_ID = PMAFR.PERSON_MASTER_ID
LEFT JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER AS ADM WITH (NOLOCK) ON PMAFR.ADDRESS_MASTER_ID = ADM.ADDRESS_MASTER_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND COALESCE(CB1.SENDERACNO,CB1.BENEFICIARYACNO) = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID;'''

##ccr population

ccr='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,
CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,
AD.ID_,
'CCR' AS TRAN_TYPE,
NULL AS CR_DR_DEP_WITH,
CB1.TRANSACTIONID,
CAST(CB1.DATEOFDETECTION AS DATE) AS TRAN_DATE,
CB1.TOTALAMOUNT AS TRAN_AMNT,
ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_CCR AS CB1 WITH (NOLOCK) ON CPSRD.REPORT_ID = CB1.REPORTID
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND CB1.ACCOUNTNUMBER = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID	
LEFT JOIN 
(SELECT REF.* FROM (SELECT A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.BRANCH_CODE ORDER BY A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID) 
AS REF WHERE REF.ROWNUMB = 1) AS ADM ON CB1.BRANCHORCAMID = ADM.BRANCH_CODE;'''

##pt1 population
ptone='''
--INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
--SELECT 
--CPSRD.BATCH_ID,CPSRD.REPORT_ID,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
--CPSRD.REPORT_TYPE,NULL,
--'PT1' AS TRAN_TYPE,
--NULL AS CR_DR_DEP_WITH ,
--PT1.TRANSACTIONID,
--CAST(PT1.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
--PT1.AMOUNT AS TRAN_AMNT,
--PT1.TRANSACTIONPINCODE AS LOCATION_PINCODE,
--GETDATE() AS CREATE_DATE,
--GETDATE() AS UPDATE_DATE
--FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
--INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_PTR PT1 WITH (NOLOCK) ON  CPSRD.REPORT_ID = PT1.REPORTID
--;'''

##ntr population
ntr='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,
'NTR' AS TRAN_TYPE,
CASE 
WHEN PT1.TRANSACTIONTYPE='DEPOSIT' THEN 'Credit'
WHEN PT1.TRANSACTIONTYPE='WITHDRAWAL' THEN 'Debit' 
ELSE NULL END AS CR_DR_DEP_WITH,
PT1.TRANSACTIONID,
CAST(PT1.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
PT1.AMOUNT AS TRAN_AMNT,
ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_NTR PT1 WITH (NOLOCK) ON CPSRD.REPORT_ID = PT1.REPORTID
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND COALESCE(PT1.SENDERACNO,PT1.BENEFICIARYACNO) = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID		
LEFT JOIN 
(SELECT REF.* FROM (SELECT A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.BRANCH_CODE ORDER BY A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1
) AS ADM ON PT1.TXNBRANCHCODE = ADM.BRANCH_CODE;'''

##ts2 population
tstwo='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,
'TS2' AS TRAN_TYPE,
TS2.TRANSACTIONTYPE AS TRANSACTIONTYPE,
TS2.TRANSACTIONID, 
CAST(TS2.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
TS2.AMOUNT AS TRAN_AMNT,
CASE WHEN TS2.TRANSACTIONTYPE = 'Credit' THEN ADMB.ORIGINAL_PINCODE ELSE ADMS.ORIGINAL_PINCODE END AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSTWO TS2 WITH (NOLOCK) ON  CPSRD.REPORT_ID = TS2.REPORTID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND COALESCE(TS2.SENDERACNO,TS2.BENEFICIARYACNO) = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID		
LEFT JOIN 
(SELECT REF.* FROM (SELECT IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY IFSC_CODE ORDER BY IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1) AS ADMS ON TS2.SENDERIFSC = ADMS.IFSC_CODE
LEFT JOIN 
(SELECT REF.* FROM (SELECT IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY IFSC_CODE ORDER BY IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1) AS ADMB ON TS2.BENEFICIARYIFSC = ADMB.IFSC_CODE;'''

##ts4
tsfour='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,
'TS4' AS TRAN_TYPE,
TS4.TRANSACTIONTYPE AS TRANSACTIONTYPE,
TS4.TRANSACTIONID, 
CAST(TS4.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
TS4.TRANSACTIONAMOUNT AS TRAN_AMNT,
PD.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSFOUR TS4 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS4.REPORTID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND TS4.ACCOUNTNUMBER = AD.ACCOUNTNUMBER 
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID		
LEFT JOIN (
SELECT REF.* FROM (
SELECT POLICYNUMBER,REPORTID,BRANCHCODE,ADM.ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.REPORTID,A.BRANCHCODE, A.POLICYNUMBER ORDER BY A.REPORTID,A.BRANCHCODE, A.POLICYNUMBER) AS ROWNUMB
FROM   FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_POLICYDETAIL A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.BRANCH BRS WITH (NOLOCK) ON A.BRANCHCODE = BRS.BRANCH_CODE
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADM WITH (NOLOCK) ON BRS.ADDRESS_MASTER_ID = ADM.ADDRESS_MASTER_ID
) AS REF WHERE ROWNUMB = 1) AS PD ON TS4.POLICYNUMBER = PD.POLICYNUMBER AND TS4.REPORTID = PD.REPORTID 
AND TS4.TRANSACTIONID IS NOT NULL;'''

##ts5
tsfive='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT, 
CPSRD.REPORT_TYPE,AD.ID_,'TS5' AS TRAN_TYPE,
CASE WHEN TS5.TRANSACTIONTYPE IN ('SIP','PURCHASE') THEN 'Credit' WHEN TS5.TRANSACTIONTYPE IN ('SWP','REDEMPTION') THEN 'Debit' END AS TRANSACTIONTYPE,
TS5.TRANSACTIONID, CAST(TS5.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
TS5.AMOUNT AS TRAN_AMNT,
ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSFIVE TS5 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS5.REPORTID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND TS5.ACCOUNTNO = AD.ACCOUNTNUMBER 
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID	
LEFT JOIN (
SELECT REF.* FROM (SELECT POLICYNUMBER,REPORTID,BRANCHCODE,ADM.ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.REPORTID,A.BRANCHCODE, A.POLICYNUMBER ORDER BY A.REPORTID,A.BRANCHCODE, A.POLICYNUMBER) AS ROWNUMB
FROM   FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_POLICYDETAIL A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.BRANCH BRS WITH (NOLOCK) ON A.BRANCHCODE = BRS.BRANCH_CODE
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADM WITH (NOLOCK) ON BRS.ADDRESS_MASTER_ID = ADM.ADDRESS_MASTER_ID
) AS REF WHERE ROWNUMB = 1) AS ADM ON TS5.ACCOUNTNO = ADM.POLICYNUMBER AND TS5.REPORTID = ADM.REPORTID
WHERE  TS5.TRANSACTIONID IS NOT NULL;'''

##ts6  Function team has written that Transaction Type and Location cannot be determined
tssix='''
--INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
--SELECT 
--CPSRD.BATCH_ID,
--CPSRD.REPORT_ID,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
--CPSRD.REPORT_TYPE,NULL,
--'TS6' AS TRAN_TYPE,
--NULL AS TRANSACTIONTYPE,
--TS6.TRANSACTIONID, 
--CAST(TS6.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
--TS6.AMOUNT AS TRAN_AMNT,
--NULL AS LOCATION_PINCODE,
--GETDATE() AS CREATE_DATE,
--GETDATE() AS UPDATE_DATE
--FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
--INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSSIX TS6 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS6.REPORTID
--'''

##ts7
tsseven='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,'TS7' AS TRAN_TYPE,CASE WHEN TS7.TRANSACTIONTYPE ='BUY' THEN 'Credit'
WHEN TS7.TRANSACTIONTYPE ='SELL' THEN 'Debit'
END AS TRANSACTIONTYPE,
TS7.TRANSACTIONID, CAST(TS7.TRADEDATE AS DATE) AS TRAN_DATE,
TS7.AMOUNT AS TRAN_AMNT,ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSSEVEN TS7 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS7.REPORTID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND TS7.RELATEDBANKACNO = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID	
LEFT JOIN (
SELECT REF.* FROM (
SELECT ACCOUNTNUMBER,REPORTID,BRANCHCODE,ADM.ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.REPORTID,A.BRANCHCODE, A.ACCOUNTNUMBER ORDER BY A.REPORTID,A.BRANCHCODE, A.ACCOUNTNUMBER) AS ROWNUMB
FROM   FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.BRANCH BRS WITH (NOLOCK) ON A.BRANCHCODE = BRS.BRANCH_CODE
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADM WITH (NOLOCK) ON BRS.ADDRESS_MASTER_ID = ADM.ADDRESS_MASTER_ID
) AS REF WHERE ROWNUMB = 1) AS ADM ON TS7.DPID = ADM.ACCOUNTNUMBER AND TS7.REPORTID = ADM.REPORTID;'''

##ts8
tseight='''
--INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
--SELECT 
--CPSRD.BATCH_ID,CPSRD.REPORT_ID,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
--CPSRD.REPORT_TYPE,COALESCE(ADS.ID_,ADB.ID_) ,
--'TS8' AS TRAN_TYPE,
--NULL AS TRANSACTIONTYPE,
--TS8.TRANSACTIONID, 
--CAST(TS8.TRADEDATE AS DATE) AS TRAN_DATE,
--TS8.AMOUNT AS TRAN_AMNT,
--'' AS LOCATION_PINCODE,
--ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
--GETDATE() AS CREATE_DATE,
--GETDATE() AS UPDATE_DATE
--FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
--INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSEIGHT TS8 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS8.REPORTID
--
--INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
--(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
--K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
--FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
--INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
--
--WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
--UNION 
--SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
--K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
--FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
--INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
--
--WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
--) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID		
--LEFT JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL ADS WITH (NOLOCK) ON TS8.DPIDSELLER = ADS.ACCOUNTNUMBER
--LEFT JOIN FINCORE_DB.FINCORE.BRANCH BRS WITH (NOLOCK) ON ADS.BRANCHCODE = BRS.BRANCH_CODE
--LEFT JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON BRS.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
--LEFT JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL ADB WITH (NOLOCK) ON TS8.DPIDBUYER = ADB.ACCOUNTNUMBER
--LEFT JOIN FINCORE_DB.FINCORE.BRANCH BRB WITH (NOLOCK) ON ADB.BRANCHCODE = BRB.BRANCH_CODE
--LEFT JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMB WITH (NOLOCK) ON BRB.ADDRESS_MASTER_ID = ADMB.ADDRESS_MASTER_ID;'''

##TS9  
tsnine='''
--INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
--SELECT 
--CPSRD.BATCH_ID,CPSRD.REPORT_ID,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
--CPSRD.REPORT_TYPE,NULL,
--'TS9' AS TRAN_TYPE,
--CASE WHEN TS9.BOIDFLAG='BUYER' THEN 'Credit'
--WHEN TS9.BOIDFLAG='SELLER' THEN 'Debit'
--END AS TRANSACTIONTYPE,
--TS9.TRANSACTIONID, 
--CAST(TS9.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
--TS9.AMOUNT AS TRAN_AMNT,
--ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
--GETDATE() AS CREATE_DATE,
--GETDATE() AS UPDATE_DATE
--FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
--INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSNINE TS9 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS9.REPORTID
--;'''

##ts10
tsten='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,
CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,
AD.ID_,
'TS10' AS TRAN_TYPE,
CASE WHEN TS10.RELATIONFLAG='SENDER' THEN 'Debit'
WHEN TS10.RELATIONFLAG='RECIEVER' THEN 'Credit'
END AS TRANSACTIONTYPE,
TS10.TRANSACTIONID, 
CAST(TS10.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
TS10.AMOUNT AS TRAN_AMNT,
ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN  FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSTEN TS10 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS10.REPORTID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND TS10.ACCOUNTNUMBER = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID
WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID																																																																																																										
LEFT JOIN (SELECT REF.* FROM (SELECT A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
,ROW_NUMBER() OVER (PARTITION BY A.BRANCH_CODE ORDER BY A.BRANCH_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
) AS REF WHERE REF.ROWNUMB = 1
) AS ADM ON TS10.TRANSACTIONBRANCHID = ADM.BRANCH_CODE;'''

##ts11
tseleven='''
--INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
--SELECT 
--CPSRD.BATCH_ID,
--CPSRD.REPORT_ID,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
--CPSRD.REPORT_TYPE,NULL,
--'TS11'AS TRAN_TYPE,
--CASE 
--WHEN TS11.TRANSACTIONTYPE IN ('ATM', 'ATM CASH WITHDRAWAL', 'CHARGE' ) THEN 'Debit'
--WHEN TS11.TRANSACTIONTYPE IN ('CASH DEPOSIT AT ATM', 'REVERSAL' ) THEN 'Credit'
--END AS TRANSACTIONTYPE,
--TS11.TRANSACTIONID, 
--CAST(TS11.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
--TS11.AMOUNTININR AS TRAN_AMNT,
--NULL AS LOCATION_PINCODE,
--GETDATE() AS CREATE_DATE,
--GETDATE() AS UPDATE_DATE
--FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
--INNER JOIN   FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSELEVEN TS11 WITH (NOLOCK) ON CPSRD.BATCH_ID= TS11.BATCHID AND CPSRD.REPORT_ID = TS11.REPORTID
--;'''

##ts12
tstwelve='''
--INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
--SELECT 
--CPSRD.BATCH_ID,CPSRD.REPORT_ID,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
--CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
--CPSRD.REPORT_TYPE,NULL,
--'TS12'AS TRAN_TYPE,
--CASE WHEN TS12.TRANSACTIONTYPE ='PAY-IN' THEN 'Credit'
--WHEN TS12.TRANSACTIONTYPE ='PAY-OUT' THEN 'Debit'
--END AS TRANSACTIONTYPE,
--TS12.TRANSACTIONID, 
--CAST(TS12.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
--TS12.AMOUNT AS TRAN_AMNT,
--ADM.ORIGINAL_PINCODE AS LOCATION_PINCODE,
--GETDATE() AS CREATE_DATE,
--GETDATE() AS UPDATE_DATE
--FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
--INNER JOIN  FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_TSTWELVE TS12 WITH (NOLOCK) ON CPSRD.REPORT_ID = TS12.REPORTID
--
--LEFT JOIN 
--(
--SELECT REF.* FROM (
--SELECT A.IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE
--,ROW_NUMBER() OVER (PARTITION BY A.IFSC_CODE ORDER BY A.IFSC_CODE, A.ADDRESS_MASTER_ID,ORIGINAL_PINCODE ) AS ROWNUMB
--FROM FINCORE_DB.FINCORE.BRANCH A WITH (NOLOCK)
--INNER JOIN FINCORE_DB.FINCORE.ADDRESS_MASTER ADMS WITH (NOLOCK) ON A.ADDRESS_MASTER_ID = ADMS.ADDRESS_MASTER_ID
--) AS REF WHERE REF.ROWNUMB = 1
--) AS ADM ON TS12.IFSC = ADM.IFSC_CODE;'''

##gen reporting
genreporting='''
INSERT INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG
SELECT DISTINCT
CPSRD.BATCH_ID,
CPSRD.REPORT_ID,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,4,4))) = 4 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,4,4) END   AS YEAR_REPORT,
CASE WHEN LEN(TRIM(SUBSTRING(CPSRD.REPORTING_PERIOD,1,2))) = 2 THEN SUBSTRING(CPSRD.REPORTING_PERIOD,1,2) END AS MONTH_REPORT,
CPSRD.REPORT_TYPE,AD.ID_,
'GT1'AS TRAN_TYPE,
NULL AS TRANSACTIONTYPE,
FGR.TRANSACTIONID, 
CAST(FGR.TRANSACTIONDATE AS DATE) AS TRAN_DATE,
FGR.TRANSACTIONAMOUNT AS TRAN_AMNT,
NULL AS LOCATION_PINCODE,
ACCNT.ENTITY_ID ,ACCNT.[ENTITY_TYPE],NULL,NULL,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE
FROM  [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID =  CPSRD.REPORT_ID
INNER JOIN   FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_GENREPORTING FGR WITH (NOLOCK) ON CPSRD.REPORT_ID = FGR.REPORTID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_ACCOUNTDETAIL AD WITH (NOLOCK) ON CPSRD.REPORT_ID = AD.REPORTID AND FGR.SENDERACNO = AD.ACCOUNTNUMBER
INNER JOIN (SELECT REF.BATCH_ID,REF.REPORT_ID,REF.FINGATE_ACCOUNT_ID,REF.PERSON_MASTER_ID AS ENTITY_ID,REF.ENTITY_TYPE FROM
(SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.PERSON_MASTER_ID,'I' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION
FROM FINCORE_DB.FINCORE.PERSON_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID

WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) 
UNION 
SELECT DISTINCT K.BATCH_ID,K.REPORT_ID,C.FINGATE_ACCOUNT_ID,C.ORGANIZATION_MASTER_ID AS ENTITY_ID,'O' AS ENTITY_TYPE,
K.PROCESSING_FLAG_CPS2_ACCOUNT,K.PROCESSING_FLAG_CPS2_ADDRESS,K.PROCESSING_FLAG_CPS2_CONTACT,K.PROCESSING_FLAG_CPS2_INDIVIDUAL,K.PROCESSING_FLAG_CPS2_ORGANIZATION 
FROM FINCORE_DB.FINCORE.ORGANIZATION_MASTER_ACCOUNT_FINGATE_REPORT C WITH (NOLOCK)
INNER JOIN FINCORE_DB.FINCORE.CPS_REPORT_DETAILS K WITH (NOLOCK) ON C.REPORT_ID=K.REPORT_ID

WHERE C.ACCOUNT_RELATIONSHIP_TYPE IN ( 'ACCOUNT HOLDER','PROPRIETOR','LIFE ASSURED','BENEFICIAL OWNER' ) ) AS REF 
) AS ACCNT ON CPSRD.REPORT_ID=ACCNT.REPORT_ID AND AD.ID_ = ACCNT.FINGATE_ACCOUNT_ID;'''


In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
li=[tsone,tctwo,tsthree,tcone,cbone,ccr,ptone,ntr,tstwo,tsfour,tsfive,tssix,tsseven,tseight,tsnine,tsten,tseleven,tstwelve,genreporting]
for i in li:
    cursor.execute(i)
    print('Done')
    conn.commit()

In [ ]:
cursor.execute(f'''UPDATE FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN SET JOB_END_DATE = GETDATE(),JOB_STATUS = 'SUCCESS',NO_OF_RECORD = {no_of_records} ,SOURCE_MAX_DATE =  (SELECT MAX(CREATE_DATE) FROM FINCORE_DB.FINCORE.CPS_REPORT_DETAILS WITH (NOLOCK)) WHERE  JOB_RUN_ID ={job_id} AND JOB_NAME='TRANSACTION_SUMMARY_STG_PROCESSING' ''')
conn.commit()

In [ ]:
print('SUMMARY_STG_INSERTION_COMPLETED')